<a href="https://colab.research.google.com/github/abdullah-farooqi/Deep_learning/blob/main/bee_vs_wasp_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
path = '../input/bee-vs-wasp/kaggle_bee_vs_wasp'
label = os.path.join(path, 'labels.csv')
df = pd.read_csv(label)

In [ ]:
df.head()

In [ ]:
df["label"].value_counts()

In [ ]:
df = df[df["label"].isin(["wasp", "bee", "insect", "other"]) & (df["photo_quality"] == 1)]

In [ ]:
def split_data(data):

    if data['is_validation']==1:
        return 'validation'
    if data['is_validation']==0 and data['is_final_validation']==0:
        return 'train'
    else:
        return 'test'

In [ ]:
df['dataset_type'] = df.apply(split_data, axis=1)

In [ ]:
df["path"] = df["path"].apply(lambda x: os.path.join(path, x.replace("\\", "/")))


In [ ]:
df.head()

In [ ]:
train_df = df[df['dataset_type'] == 'train'][["path", "label"]]
test_df = df[df['dataset_type'] == 'test'][["path", "label"]]
val_df = df[df['dataset_type'] == 'validation'][["path", "label"]]

In [ ]:
train_df

In [ ]:
num_classes = df['label'].nunique()


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.2,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="path",
    y_col="label",
    target_size=(256, 256),
    batch_size=32,
    class_mode="categorical",
    shuffle=True
)

validation_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col="path",
    y_col="label",
    target_size=(256, 256),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="path",
    y_col="label",
    target_size=(256, 256),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', padding='valid', input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(64, (3, 3), activation='relu', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Conv2D(128, (3, 3), activation='relu', padding='valid'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid'))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(
    "best_model.h5",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True,
    verbose=1
)



In [ ]:
history = model.fit(train_generator,validation_data=validation_generator,epochs=50,callbacks=[early_stop,checkpoint])


In [ ]:
history2 = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,  # total desired epochs
    initial_epoch=50,  # resume from where you left off
    callbacks=[early_stop, lr_reduce]
)


In [ ]:
pred_probs = model.predict(test_generator)
pred_labels = (pred_probs > 0.5).astype(int).flatten()  # Convert probabilities to 0 or 1

